### Feature Engineering

To understand how linear regression works, we've stuck to using features from the training dataset that contained no missing values and were already in convenient numeric representation. In this notebook, we'' explore how to transform some of the remaining features so we can use them in our model. Broadly, the process of processing and creating new features is known as [feature engineering](https://en.wikipedia.org/wiki/Feature_engineering). Feature engineering is a bit of an art and having knowledge  in the specific domain (in this case real estate) can help you create better features. In this mission, we'' focus on some domain-independent strategies that work for all problems.

In the first half of this mission, we'll focus only on columns that contain no missing values but still aren't in the proper format to use in a linear regression model. In the latter half of this mission, we'll explore some ways to deal with missing values.

Amongst the columns that don't contain missing values, some of the common issues include:
- The column is not numerical (e.g. a zoning code represented using text)
- The column is numerical but not ordinal (e.g. zip code values)
- The column is numerical but isn't representative of the type of relationship with the target column (e.g. year values)

Let's start by filtering the training set to just the columns containing no missing values.

#### Instructions
- Select just the columns from the train data frame that contain no missing values.
- Assign the resulting data frame, that contains just these columns, to df_no_mv.
- Use the variables display to become familiar with these columns.

In [15]:
import pandas as pd
data = pd.read_csv('AmesHousing.txt', delimiter="\t")
target='SalePrice'
total_records = len(data)
total_records

2930

In [16]:
# assign approx 50% of the data to train and 50% to test
train = data[0:1460]
test = data[1460:]

train_null_counts = train.isnull().sum()
no_mv_cols = train_null_counts[train_null_counts == 0]

df_no_mv = train[no_mv_cols.index]
df_no_mv.head(4)

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000


Note that some of the columns in the data frame df_no_mv contain string values. If these columns contain only a limited set of unique values, they are known as categorical features. As the name suggests, a categorical feature groups a specific training example into a specific category. Here are some examples from the dataset:


In [17]:
train['Utilities'].value_counts()

AllPub    1457
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64

In [18]:
train['Street'].value_counts()

Pave    1455
Grvl       5
Name: Street, dtype: int64

In [19]:
train['House Style'].value_counts()

1Story    743
2Story    440
1.5Fin    160
SLvl       60
SFoyer     35
2.5Unf     11
1.5Unf      8
2.5Fin      3
Name: House Style, dtype: int64

To use these features in our model, we need to transform them into numerical representations. Thankfully, pandas makes this easy because the library has a special [categorical data type](https://pandas.pydata.org/pandas-docs/stable/categorical.html). We can convert any column that contains no missing values to the categorical data type using the pandas.Series.astype() method.

```
>>> train['Utilities'] = train['Utilities'].astype('category')
```

When a column is converted to the categorical data type, pandas assigns a code to each unique value in the common. Unless we access these values directly, most of the pandas manipulation operations that work for string columns will work for categorical ones as well.

We need to use the .cat accessor followed by the .codes property to actually access the underlying numerical representation of a column:

```
>>> train['Utilities'].cat.codes
```

#### Instructions

- Convert all of the text columns in train to the categorical data type.
- Select the Utilities column, return the categorical codes, and display the unique value counts for those codes: - train['Utilities'].cat.codes.value_counts()

In [20]:
text_cols = df_no_mv.select_dtypes(include=['object']).columns
for col in text_cols:
    train[col] = train[col].astype('category')

train['Utilities'].cat.codes.value_counts()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    1457
2       2
1       1
dtype: int64

In [21]:
train['Utilities'].unique()

[AllPub, NoSewr, NoSeWa]
Categories (3, object): [AllPub, NoSewr, NoSeWa]

When we convert a column to the categorical data type, pandas assigns a number from 0 to n-1 (where n is the number of unique values in a column) for each value. The drawback with this approach is that one of the assumptions of linear regression is violated here. Linear regression operates under the assumption that the features are linearly correlated with the target column. For a categorical feature, however, there's no actual numerical meaning to the categorical codes that pandas assigned for that colum. An increase in the Utilities column from 1 to 2 has no correlation value with the target column, and the categorical codes are instead used for uniqueness and exclusivity (the category associated with 0 is different than the one associated with 1).

The common solution is to use a technique called dummy coding. Instead of having a single column with n integer codes, we have n binary columns. Here's what that would look like for the Utilities column:

| Utilities_AllPub | Utilities_NoSewr | Utilities_NoSeWa |
|:----------------:|:----------------:|:----------------:|
|    1             |       0          |       0          |
|    1             |       0          |       0          |
|    1             |       0          |       0          |
|    1             |       0          |       0          |

Because the original values for the first 4 rows were AllPub, in the new scheme, they contain the binary value for true (1) in the Utilities_AllPub column and 0 for the other 2 columns.

Pandas thankfully has a convenience method to help us apply this transformation for all of the text columns called [pandas.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html):

```
dummy_cols = pd.get_dummies()
```

#### Instructions
- Convert all of the columns in text_cols from the train data frame into dummy columns.
- Delete the original columns from text_cols from the train data frame.

In [22]:
for col in text_cols:
    col_dummies = pd.get_dummies(train[col])
    train = pd.concat([train,col_dummies],axis=1)
    del train[col]

In the last few screens, we focused on categorical values that were represented as text columns. Some of the numerical columns in the data set are also categorical and only have a limited set of unique values. We won't explicitly explore those coumns in this mission, but the feature transformation process is the same if the numbers used in those categories have no numerical meaning.

Let's now look at numerical features that aren't categorical, but whose numerical representation needs to be improved. We'll focus on the Year Remod/Add and Year Built columns:

```
>>> train[['Year Remod/Add', 'Year Built']]
```
| 0  | 1960 | 1960 |
|:--:|:----:|:----:|
|1   |1961  | 1961 |
|2   |1958  | 1958 |
|3   |1968  | 1968 |
|4   |1998  | 1997 |  
...
The two main issues with these features are:

Year values aren't representative of how old a house is
The Year Remod/Add column doesn't actually provide useful information for a linear regression model
The challenge with year values like 1960 and 1961 is that they don't do a good capture how old a house is. For example, a house that was built in 1960 but sold in 1980 was sold in half the time one built in 1960 and sold in 2000. Instead of the years certain events happened, we want the difference between those years. We should create a new column that's the difference between both of these columns.

For this particular piece of information (years until remodeled), this is a sensible approach. Domain knowledge can help you understand how to best transform features to represent information well for a linear model. If you're ever confused about a feature or how it should be represented, reading scientific papers or posts by researchers in the specific domain is critical. Many winners of Kaggle data science competitions, for example, claim that their focus on data preparation and feature engineering combined with common machine learning models helped them win.

#### Instructions
- Create a new column years_until_remod in the train data frame that represents the difference between Year Remod/Add (the later value) and Year Built (the earlier value).

In [23]:
train["years_until_remod"] = train['Year Remod/Add'] - train['Year Built']

In the next few screens, we'll focus on handling columns with missing values. When values are missing in a column, there are two main approaches we can take:

- Remove rows containing missing values for specific columns
  - Pro: Rows containing missing values are removed, leaving only clean data for modeling
  - Con: Entire observations from the training set are removed, which can reduce overall prediction accuracy
- Impute (or replace) missing values using a descriptive statistic from the column
  - Pro: Missing values are replaced with potentially similar estimates, preserving the rest of the observation in the model.
  - Con: Depending on the approach, we may be adding noisy data for the model to learn
  
Given that we only have 1460 training examples (with ~80 potentially useful features), we don't want to remove any of these rows from the dataset. Let's instead focus on imputation techniques.

We'll focus on columns that contain at least 1 missing value but less than 365 missing values (or 25% of the number of rows in the training set). There's no strict threshold, and many people instead use a 50% cutoff (if half the values in a column are missing, it's automatically dropped). Having some domain knowledge can help with determining an acceptable cutoff value.

#### Instructions

- Select only the columns from train that contain more than 0 missing values but less than 584 missing values. Assign the resulting data frame to df_missing_values.
- Display the number of missing values for each column in df_missing_values.
- Display the data type for each column in df_missing_values.

In [24]:
import pandas as pd

data = pd.read_csv('AmesHousing.txt', delimiter="\t")
train = data[0:1460]
test = data[1460:]

train_null_counts = train.isnull().sum()

df_missing_values = train[train_null_counts[ (train_null_counts > 0) & (train_null_counts<584)].index]

print(df_missing_values.isnull().sum())
print(df_missing_values.dtypes)



Lot Frontage      249
Mas Vnr Type       11
Mas Vnr Area       11
Bsmt Qual          40
Bsmt Cond          40
Bsmt Exposure      41
BsmtFin Type 1     40
BsmtFin SF 1        1
BsmtFin Type 2     41
BsmtFin SF 2        1
Bsmt Unf SF         1
Total Bsmt SF       1
Bsmt Full Bath      1
Bsmt Half Bath      1
Garage Type        74
Garage Yr Blt      75
Garage Finish      75
Garage Qual        75
Garage Cond        75
dtype: int64
Lot Frontage      float64
Mas Vnr Type       object
Mas Vnr Area      float64
Bsmt Qual          object
Bsmt Cond          object
Bsmt Exposure      object
BsmtFin Type 1     object
BsmtFin SF 1      float64
BsmtFin Type 2     object
BsmtFin SF 2      float64
Bsmt Unf SF       float64
Total Bsmt SF     float64
Bsmt Full Bath    float64
Bsmt Half Bath    float64
Garage Type        object
Garage Yr Blt     float64
Garage Finish      object
Garage Qual        object
Garage Cond        object
dtype: object


It looks like about half of the columns in df_missing_values are string columns (object data type), while about half are float64 columns. For numerical columns with missing values, a common strategy is to compute the mean, median, or mode of each column and replace all missing values in that column with that value.

Because imputation is a common task, pandas contains a method named pandas.DataFrame.fillna() that we can use for this. If we pass in a value, all of the missing values (NaN) in the data frame are replaced by that value.

#### Instructions
- Impute the missing values from float_cols with the column's mean.
- Check for any missing values in float_cols.

In [25]:
float_cols = df_missing_values.select_dtypes(include=['float'])
float_cols = float_cols.fillna(df_missing_values.mean())
print(float_cols.isnull().sum())

Lot Frontage      0
Mas Vnr Area      0
BsmtFin SF 1      0
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Bsmt Full Bath    0
Bsmt Half Bath    0
Garage Yr Blt     0
dtype: int64
